In [56]:
import dash
# import dash_auth
# from users import USERNAME_PASSWORD_PAIRS
from dash import dcc
from dash import html
import plotly
import dash_bootstrap_components as dbc
# import dash_daq as dq
from jupyter_dash import JupyterDash
from dash import Dash
import pandas as pd  
import joblib
import base64, io, os
from dash import  dash_table
import dash
import pandas as pd
from dash import dash_table
import pickle
from dash.dependencies import Input, Output, State, ALL, MATCH
import json
# from functools import reduce
from dash import Input, Output, State, html
from tqdm.notebook import trange, tqdm
import pickle
import numpy as np
import pandas as pd
# from cryptography.fernet import Fernet
import json
import io, base64, os
import pandas as pd
import datetime
import time
import re
import plotly.express as px
import plotly.graph_objects as go

import geopandas
import tifffile as tiff
import cv2
import matplotlib; matplotlib.rcParams['figure.dpi']=300
import matplotlib.pyplot as plt
import pickle
import numpy as np
from pathpretrain.utils import generate_tissue_mask
import scipy.ndimage as ndimage
from skimage.util import compare_images
from skimage.measure import regionprops_table
from skimage import exposure
import tempfile
import matplotlib.image as mpimg
from skimage import data, draw
# from scipy import ndimage

In [2]:
with open("./data/match_dict.pkl", "rb") as file:
    match_dict = pickle.load(file)

annotation_data = geopandas.read_file("./data/_SS12251_140745.geojson")
im_large = tiff.imread("./data/_SS12251_140745.svs")
im_small = cv2.resize(im_large, None, fx=1/15, fy=1/15)

with open("./data/011320_002_N_HR_dict.pkl", "rb") as input_file:
    metal_data = pickle.load(input_file)

In [3]:
im_medium = cv2.resize(im_large, None, fx=1/7, fy=1/7)

In [4]:
im_small.shape

(3101, 4657, 3)

In [5]:
metal_data['metals']['Ca44'].shape

(871, 1499)

In [6]:
def crop_slide(image_array):
#     slide_file = list(filter(lambda x: match_dict[x] == imageId, match_dict))[0]    
    im_large=image_array#tifffile.imread(f"/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/metals/breast/new_slide_images_4_6_23/{slide_file}.svs")

    tissue_mask=generate_tissue_mask(im_large*5,compression=32.,threshold=240,kernel=10,gray_before_close=True) 
    compression=32
    
    # Metal image is only portion of slide
#     if (imageId == "092619_007_T_HER2"):    
        
#     if (imageId == "102720_003_T_TNBC"):
#         labels_small=ndimage.label(cv2.resize(tissue_mask.astype(np.uint8),None,fx=1/compression,fy=1/compression,interpolation=cv2.INTER_NEAREST))[0]
#         lbl_table=pd.DataFrame(regionprops_table(labels_small,properties=['bbox','area',"label"])).sort_values("area",ascending=False)
#         xmin,ymin,xmax,ymax=(lbl_table.iloc[1][[col for col in lbl_table if "bbox" in col]]*compression).astype(int)   
#     else:  
    labels_small=ndimage.label(cv2.resize(tissue_mask.astype(np.uint8),None,fx=1/compression,fy=1/compression,interpolation=cv2.INTER_NEAREST))[0]
    lbl_table=pd.DataFrame(regionprops_table(labels_small,properties=['bbox','area',"label"])).sort_values("area",ascending=False)
    xmin,ymin,xmax,ymax=(lbl_table.iloc[0][[col for col in lbl_table if "bbox" in col]]*compression).astype(int)   
    
    im_large_crop=im_large[xmin:xmax,ymin:ymax].copy()
    im_large_crop[~tissue_mask[xmin:xmax,ymin:ymax]]=255
    
    return im_large_crop, [[xmin, ymin], [xmax, ymax]], tissue_mask

In [7]:
im_large_crop, minmax_list, tissue_mask = crop_slide(im_large)
im_small_crop = cv2.resize(im_large_crop, None, fx=1/15, fy=1/15)

hne_image = px.imshow(im_small_crop)
metal_image = px.imshow(np.log(metal_data['metals']['Ca44'])+10, color_continuous_scale='jet')

/Users/luyunrui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning:

divide by zero encountered in log

/Users/luyunrui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning:

invalid value encountered in log



In [8]:
annotation_data['annot']=annotation_data['classification'].map(lambda x: x.get("name","none"))

gp2=annotation_data.copy()
gp2['geometry']=gp2['geometry'].scale(1/7,1/7,origin=(0,0))
gp2['coords'] = gp2['geometry'].apply(lambda x: x.representative_point().coords[:])
gp2['coords'] = [coords[0] for coords in gp2['coords']]

In [45]:
app = JupyterDash(__name__, 
                  external_stylesheets=[dbc.themes.SPACELAB], 
                 )

markdown_text_title = '''
# **Metals Demo**
'''

tab_Data_content = dbc.Card(
    dbc.CardBody(
        [
            html.P("This is Data tab", className="card-text"),
        ]
    ),
    className="mt-3",
)

image_no_axis_layout = go.Layout(
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
)

hne_image.update_layout(image_no_axis_layout)
metal_image.update_layout(image_no_axis_layout)
hne_image.update_layout(title=dict(text="H&E Image", x=0.5, y=0.95, xanchor='center'))

def update_metal_image(selected_metal, 
                       np_log, 
                      ):
    if np_log == 'With np log transform':
        metal_image_data = np.log(metal_data['metals'][selected_metal]) + 10
    else:
        metal_image_data = metal_data['metals'][selected_metal]
    metal_image = px.imshow(metal_image_data, color_continuous_scale='jet')
    metal_image.update_layout(image_no_axis_layout)
    metal_image.update_layout(title=dict(text="Metal Image - {}".format(selected_metal), x=0.5, y=0.95, xanchor='center'))
    return metal_image

tab_Visualization_content = dbc.Card(
    dbc.CardBody(
        [
            dbc.Row(
            [
                dbc.Col(dcc.Graph(id="hne_image", figure=hne_image, 
                                 ), 
                        width=6),
                dbc.Col(dcc.Graph(id="metal_image", figure=metal_image, 
                                 ), 
                        width=6),
            ]
        ),
            dbc.Row([
                dcc.Dropdown(
                            id='metal_dropdown',
                            options=list(metal_data['metals'].keys()),
                            value='Ca44',
                            searchable=True, 
                            style={'width': '110px'},
                                ),
                dcc.Dropdown(
                            id='np_log_dropdown',
                            options=['With np log transform', 'Without np log transform'],
                            value='With np log transform',
                            style={'width': '300px'},
                                ),
            ], justify="end",),
        ]
    ),
    className="mt-3",
)

tab_Coregister_content = dbc.Card(
    dbc.CardBody(
        [
            html.P("This is Coregister tab", className="card-text"),
        ]
    ),
    className="mt-3",
)

original_image = px.imshow(im_medium)
original_image.update_layout(image_no_axis_layout)
original_image.update_layout(title=dict(text="H&E Image", x=0.5, y=0.95, xanchor='center'))
original_image.update_layout(margin={'l': 0, 'r': 0, 't': 0, 'b': 0})

metal_annotation_image = px.imshow(np.log(metal_data['metals']['Ca44'])+10)
metal_annotation_image.update_layout(margin={'l': 0, 'r': 0, 't': 0, 'b': 0})
metal_annotation_image.update_layout(image_no_axis_layout)
metal_annotation_image.update(layout_coloraxis_showscale=False)
metal_annotation_image.update_traces(hovertemplate=None, hoverinfo='skip')
metal_annotation_image.update_layout(
#     newshape_line_color=color_dict[DEFAULT_ATYPE],
    dragmode="drawrect",
)

image_annotation_card = dbc.Card(
    children = [dbc.CardHeader(html.H2("Annotation area")),
        dbc.CardBody(
            [
                dcc.Graph(
                    id="annotate_metal_image",
                    figure=metal_annotation_image,
                    config={"modeBarButtonsToAdd": ["drawrect", "eraseshape", "drawclosedpath",]},
#                     config={"modeBarButtonsToAdd": ["drawrect", "eraseshape"]},
                )
            ]
        ),
        dbc.CardFooter(
            [
                dcc.Markdown(
                    "To annotate the above image"
                    "rectangle .\n\n"
                    "**Choose metals**:"
                ),
                dbc.ButtonGroup(
                    [
                        dbc.Button("Previous image", id="previous", outline=True),
                        dbc.Button("Next image", id="next", outline=True),
                    ],
                    size="lg",
                    style={"width": "100%"},
                ),
#                 dbc.Row([
#                     dcc.Dropdown(
#                                 id='metal_dropdown_annotation',
#                                 options=list(metal_data['metals'].keys()),
#                                 value='Ca44',
#                                 searchable=True, 
#                                 style={'width': '110px'},
#                                     ),
#                     dcc.Dropdown(
#                                 id='np_log_dropdown',
#                                 options=['With np log transform', 'Without np log transform'],
#                                 value='With np log transform',
#                                 style={'width': '300px'},
#                                     ),
#                 ], justify="end",),
            ]
        ),]
)

annotated_data_card = dbc.Card(
    children = [dbc.CardHeader(html.H2("Table")),
        dbc.CardBody(
            [
                html.P("This is Coregister tab", className="card-text"),
            ]
        ),
        dbc.CardFooter(
            [
                dcc.Markdown(
                    "To annotate the above image"
                    "rectangle .\n\n"
                    "**Choose metals**:"
                ),
            ]
        ),]
)

image_annotation_table = dbc.Row(
                                [
                                    dbc.Col(dcc.Graph(figure=original_image), width=4),
                                    dbc.Col(image_annotation_card, width=5),
                                    dbc.Col(annotated_data_card, md=3),
                                    ], 
justify="start",)

tab_Measure_content = dbc.Card(
    dbc.CardBody(
        [
            image_annotation_table,
        ]
    ),
    className="mt-3",
)

all_tabs = dbc.Tabs(
    [
        dbc.Tab(tab_Data_content, label="Data"),
        dbc.Tab(tab_Visualization_content, label="Visualization"),
        dbc.Tab(tab_Coregister_content, label="Co-register"),
        dbc.Tab(tab_Measure_content, label="Measure"),
    ]
)

app.layout = html.Div([dcc.Markdown(children=markdown_text_title),
                       all_tabs])

@app.callback(
    Output('metal_image', 'figure'),
    [Input('metal_dropdown', 'value'), 
     Input('np_log_dropdown', 'value')]
)

def update_metal_image_callback(selected_metal, np_log):
    return update_metal_image(selected_metal, np_log)









# @app.callback(
#     [Output("previous", "value")],
#     [
# #         Input("previous", "n_clicks"),
# #         Input("next", "n_clicks"),
#         Input("annotate_metal_image", "relayoutData"),
#     ],)

# def modify_table_entries(graph_relayoutData,):
#     print(type(graph_relayoutData))
#     print(graph_relayoutData)




/Users/luyunrui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning:

divide by zero encountered in log

/Users/luyunrui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning:

invalid value encountered in log



In [46]:
app.run_server(debug=False,
               mode="external",
               host='localhost', 
              )

 * Running on http://localhost:8050 (Press CTRL+C to quit)
127.0.0.1 - - [29/Jul/2023 23:07:39] "GET /_alive_ae4a517c-220a-450e-8e36-d8654c2c3d1b HTTP/1.1" 200 -


Dash app running on http://localhost:8050/


127.0.0.1 - - [29/Jul/2023 23:07:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2023 23:07:50] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2023 23:07:51] "GET /_favicon.ico?v=2.5.0 HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2023 23:07:54] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2023 23:07:55] "GET /_dash-component-suites/dash/dcc/async-markdown.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Jul/2023 23:07:55] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Jul/2023 23:07:55] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Jul/2023 23:07:55] "/Users/luyunrui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning:

divide by zero encountered in log

/Users/luyunrui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning:

invalid value encountered in log

GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - 

<class 'NoneType'>
None


127.0.0.1 - - [29/Jul/2023 23:07:56] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [29/Jul/2023 23:07:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2023 23:08:01] "POST /_dash-update-component HTTP/1.1" 500 -


...

<class 'dict'>
{'dragmode': 'drawclosedpath'}


127.0.0.1 - - [29/Jul/2023 23:08:35] "POST /_dash-update-component HTTP/1.1" 500 -


<class 'dict'>
{'shapes': [{'editable': True, 'xref': 'x', 'yref': 'y', 'layer': 'above', 'opacity': 1, 'line': {'color': '#444', 'width': 4, 'dash': 'solid'}, 'fillcolor': 'rgba(0,0,0,0)', 'fillrule': 'evenodd', 'type': 'rect', 'x0': 281.34604012160116, 'y0': 277.0981879726423, 'x1': 470.7214849614587, 'y1': 405.1266577235319}, {'editable': True, 'xref': 'x', 'yref': 'y', 'layer': 'above', 'opacity': 1, 'line': {'color': '#444', 'width': 4, 'dash': 'solid'}, 'fillcolor': 'rgba(0,0,0,0)', 'fillrule': 'evenodd', 'type': 'path', 'path': 'M1153.539990299537,397.1248783641013L1044.1823390539855,399.7921381505781L1009.5079618297862,423.7974762288699L998.8389226838788,474.47541217193043L1006.8407020433093,514.4843089690834L1036.1805596945549,541.1569068338521L1094.860274997046,557.1604655527133L1142.8709511536297,554.4932057662364L1206.8851860290745,519.8188285420372L1236.22504368032,487.8117111043147L1236.22504368032,455.8045936665923L1217.5542251749819,399.7921381505781L1198.8834066696438,

127.0.0.1 - - [29/Jul/2023 23:08:59] "POST /_dash-update-component HTTP/1.1" 500 -


<class 'dict'>
{'shapes': [{'editable': True, 'xref': 'x', 'yref': 'y', 'layer': 'above', 'opacity': 1, 'line': {'color': '#444', 'width': 4, 'dash': 'solid'}, 'fillcolor': 'rgba(0,0,0,0)', 'fillrule': 'evenodd', 'type': 'rect', 'x0': 281.34604012160116, 'y0': 277.0981879726423, 'x1': 470.7214849614587, 'y1': 405.1266577235319}, {'editable': True, 'xref': 'x', 'yref': 'y', 'layer': 'above', 'opacity': 1, 'line': {'color': '#444', 'width': 4, 'dash': 'solid'}, 'fillcolor': 'rgba(0,0,0,0)', 'fillrule': 'evenodd', 'type': 'path', 'path': 'M1153.539990299537,397.1248783641013L1044.1823390539855,399.7921381505781L1009.5079618297862,423.7974762288699L998.8389226838788,474.47541217193043L1006.8407020433093,514.4843089690834L1036.1805596945549,541.1569068338521L1094.860274997046,557.1604655527133L1142.8709511536297,554.4932057662364L1206.8851860290745,519.8188285420372L1236.22504368032,487.8117111043147L1236.22504368032,455.8045936665923L1217.5542251749819,399.7921381505781L1198.8834066696438,

In [57]:
test_path = 'M1.283762541529971,866.56260078403L83.96881592231288,863.8953409975531L81.30155613583602,789.2120669762008L59.96347784402107,781.2102876167702L-0.5,786.544807189724Z'
def path_to_indices(path):
    """From SVG path to numpy array of coordinates, each row being a (row, col) point
    """
    indices_str = [
        el.replace("M", "").replace("Z", "").split(",") for el in path.split("L")
    ]
    return np.rint(np.array(indices_str, dtype=float)).astype(np.int)
def path_to_mask(path, shape):
    """From SVG path to a boolean array where all pixels enclosed by the path
    are True, and the other pixels are False.
    """
    cols, rows = path_to_indices(path).T
    rr, cc = draw.polygon(rows, cols)
    mask = np.zeros(shape, dtype=np.bool)
    mask[rr, cc] = True
    mask = ndimage.binary_fill_holes(mask)
    return mask

In [58]:
path_to_indices(test_path)

/Users/luyunrui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



array([[  1, 867],
       [ 84, 864],
       [ 81, 789],
       [ 60, 781],
       [  0, 787]])

In [59]:
path_to_mask(test_path, metal_data['metals']['Ca44'].shape)

/Users/luyunrui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

/Users/luyunrui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-n

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])